In [5]:
import sys
import pybel
import imolecule
from glom import glom
from metatlas import create_launchpad, make_xyz_from_stored_data

sys.path.append('./malt')
# from malt.algorithm.topology import Fragmentation

LOCAL_DB_CONFIG = '/home/bkrull/.fireworks/local_db.ini'
lpad = create_launchpad(LOCAL_DB_CONFIG)

# frag = Fragmentation()
# compact = frag.weights_compact
# longrange = frag.weights_longrange

tmpdir = '/home/bkrull/Desktop/tmpdir/'

In [6]:
fw = lpad.get_fw_dict_by_id(1)

def get_trajs(fw):
    parent = glom(fw, 'launches.0.action.stored_data.parent')
    children = glom(fw, 'launches.0.action.stored_data.protonated_children')

    fbase = tmpdir + 'mol-{}-'.format(fw['fw_id'])
    with open(fbase+'parent.xyz', 'w') as f:
        for coords in parent['coords']:
            f.write(
                make_xyz_from_stored_data(parent['atom_list'], coords)
            )
            
    print 'There are {} protonated children'.format(len(children))

    for i, child in enumerate(children):
        print 'This child was protonated at {}{} with hybridization {}'.format('', '', '')
        print 'This child is predicted to fragment: {}'.format(child['likely_to_break'])
        with open(fbase+'child-{}.xyz'.format(i), 'w') as f:
            for coords in child['coords']:
                f.write(
                    make_xyz_from_stored_data(child['atom_list'], coords)
                )
                
        print ''
        
    return parent, children

In [7]:
fw = lpad.get_fw_dict_by_id(5532)

parent, children = get_trajs(fw)

There are 2 protonated children
This child was protonated at  with hybridization 
This child is predicted to fragment: True

This child was protonated at  with hybridization 
This child is predicted to fragment: True



In [9]:
import pickle
    
with open(tmpdir+'parent.pickle', 'wb') as handle:
    pickle.dump(parent, handle, protocol=pickle.HIGHEST_PROTOCOL)  
    
with open(tmpdir+'children.pickle', 'wb') as handle:
    pickle.dump(children, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
imolecule.draw(pybel.readstring('xyz', make_xyz_from_stored_data(parent['atom_list'], parent['coords'][0])), 
               format="pybel")

In [11]:
imolecule.draw(pybel.readstring('xyz', make_xyz_from_stored_data(children[1]['atom_list'], 
                                                                 children[1]['coords'][0])), 
               format="pybel")

+ Picture of molecule and where it will be protonated
+ Prediction based on simple hybridization rules. WILL IT BREAK?
+ Animation of optimization trajectory, coords and graph
+ Graph cut results. Did it break?